In [14]:
#!/usr/bin/python3

# ***PURPOSE***
# This script generates the vars.txt file (which is a subject x Subject Measures matrix) used by Smith et al. in their analysis of the HCP_500 data
# See reference: https://www.fmrib.ox.ac.uk/datasets/HCP-CCA/

# ***USAGE***
# Multiple files are needed:
# 1. a .txt file containing the names of the subject measures (SMs) to be used in the analysis
# 2. a .txt file containing the names of all subjects to be analyzed (their subject IDs)
# 3. the behavioral data from HCP
# 4. the 'restricted' data from HCP (requires special access, must request this)
# 5. the rfMRI_motion.txt file
# 6. the quarter/release info file (named varsQconf.txt)

# ***NOTE***
# Files 1, 2, 5, and 6 are included in our GitHub repo (named subject_measure_names.txt, subject_ids.txt, rfMRI_motion.txt, and varsQconf.txt, respectively)

# ***EXAMPLE USAGE ON CMD LINE***
# ./generate_vars.py column_headers.txt subjects.txt <behavioral data> <restricted data> rfMRI_motion.txt varsQconf.txt

import numpy as np
import pandas as pd
from pandas import DataFrame
from numpy import genfromtxt
import os
import sys
from pprint import pprint

cwd = os.getcwd()
inputs = os.path.abspath("__file__"+"/../../inputs")
outputs = os.path.abspath("__file__"+"/../../outputs") # NOTE CHANGE THIS TO YOUR DESIRED OUTPUT PATH!

column_headers_fp = os.path.join(inputs, 'subject_measure_names.txt')
subject_ids_fp = os.path.join(inputs, 'subject_ids.txt')
behavioral_data_fp = os.path.join(inputs, 'unrestricted_500_release.csv')
restricted_data_fp = os.path.join(inputs, 'restricted_500_release.csv')
rfMRI_data_fp = os.path.join(inputs, 'rfMRI_motion.txt')
varsQconf_fp = os.path.join(inputs, 'varsQconf.txt')

# get the column headers, and names of subjects
column_headers = [line.rstrip('\n') for line in open(os.path.join(cwd,column_headers_fp))]
subjects = [line.rstrip('.pconn.nii\n') for line in open(os.path.join(cwd,subject_ids_fp))]

# now import "behavioral" and "restricted" datasets into Pandas dataframes
behavioral_data = pd.read_csv(behavioral_data_fp)
restricted_data = pd.read_csv(restricted_data_fp)


# Now we will filter out only the rows that correspond to the subjects specified in subjects.txt
# Sanity check, making sure that the filtering occurs correctly
print('behavior shape before', behavioral_data.shape)
print('shape of restricted before', restricted_data.shape)

#filter the behavioral and restricted datasets to contain only the relevant 461 subject data
behavioral_data = behavioral_data[behavioral_data['Subject'].isin(subjects)]
restricted_data = restricted_data[restricted_data['Subject'].isin(subjects)]

print('behavior shape after', behavioral_data.shape)
print('shape of restricted after', restricted_data.shape)

behavior shape before (527, 338)
shape of restricted before (542, 198)
behavior shape after (461, 338)
shape of restricted after (461, 198)


In [15]:
# Now import the rfMRI and quarter/release (varsQconf) data
varsqconf = pd.read_csv(varsQconf_fp, names=['quarter/release'])
rfmri = pd.read_csv(rfMRI_data_fp, sep=" ", names=['rfmri_motion'])

In [16]:
# reindex so that the varsqconf has the correct subject IDs as its row labels
varsqconf.index = rfmri.index

In [17]:
# concatenate the rfMRI and varsQconf data (we will need to do this later anyway)
rfmri_varsqconf = pd.concat([rfmri, varsqconf], axis=1)

In [18]:
# get the names of column headers
behav_headers=list(behavioral_data.columns.values)
restrict_headers=list(restricted_data.columns.values)

# Make lowercase
column_headers=[element.lower() for element in column_headers]
behav_headers=[element.lower() for element in behav_headers]
restrict_headers=[element.lower() for element in restrict_headers]

In [20]:
# Now let's lets get the column names that are overlapped in each
overlap_in_behav = np.intersect1d(column_headers,behav_headers)
overlap_in_restrict = np.intersect1d(column_headers,restrict_headers)

In [21]:
# Now pull out the columns and their data
# first we will need to convert all the column headers to lowercase
behavioral_data.columns = behavioral_data.columns.str.lower()
restricted_data.columns = restricted_data.columns.str.lower()

behavioral_data_filtered_cols = behavioral_data[overlap_in_behav]
restricted_data_filtered_cols = restricted_data[overlap_in_restrict]

In [22]:
# check that all dimensions are correct before we attempt to concat the dataframes
print(behavioral_data_filtered_cols.shape)
print(restricted_data_filtered_cols.shape)
print(rfmri_varsqconf.shape)

(461, 286)
(461, 190)
(461, 2)


In [23]:
# concat the dataframes

# first reindex all of them to match rfmri_varsqconf
behavioral_data_filtered_cols.index = rfmri_varsqconf.index
restricted_data_filtered_cols.index = rfmri_varsqconf.index

vars = pd.concat([behavioral_data_filtered_cols, restricted_data_filtered_cols, rfmri_varsqconf], axis = 1)

In [24]:
# drop the duplicated 'subject' column
vars = vars.drop(columns='subject')
vars = vars.reindex(columns = column_headers)

In [25]:
# output vars.txt to the 'outputs' folder
vars.to_csv(os.path.join(outputs, "vars_500_release.txt"))